In [2]:
import pulp as lp
import pandas as pd

In [4]:
# Store results for all countries
optimization_results = {}

# Load the processed data from CSV
final_df = pd.read_csv("Final_Processed_Data.csv")

# Iterate through each country
for country in final_df["Country"].unique():
    # Filter data for the specific country
    country_df = final_df[final_df["Country"] == country]

    # Extract parameters
    advertiser_data = country_df[["Month", "Advertisers", "Avg_Budget"]].reset_index(drop=True)
    initial_agents = country_df["Existing_Agent_Count"].iloc[0]
    annual_salary = country_df["Annual_Agent_Salary_USD"].iloc[0]
    monthly_salary = annual_salary / 12

    # Constants
    firing_cost_perc = 0.4
    max_adv_per_agent = 10
    expected_uplift = 0.135
    n_months = len(advertiser_data)

    # Initialize PuLP problem
    prob = lp.LpProblem(f"Dynamic_Staffing_Optimization_{country}", lp.LpMinimize)

    # Decision Variables
    months = list(range(n_months))
    A = lp.LpVariable.dicts("Available_Agents", months, lowBound=0, cat='Integer')
    N = lp.LpVariable.dicts("New_Hires", months, lowBound=0, cat='Integer')
    F = lp.LpVariable.dicts("Fired", months, lowBound=0, cat='Integer')
    S = lp.LpVariable.dicts("Assigned_Advertisers", months, lowBound=0, cat='Integer')
    W = lp.LpVariable.dicts("Waiting_Pool", months, lowBound=0, cat='Integer')
    D = lp.LpVariable.dicts("Dropped_Advertisers", months, lowBound=0, cat='Integer')
    G = lp.LpVariable.dicts("Graduated_Advertisers", months, lowBound=0, cat='Integer')

    # Initial Conditions
    prob += A[0] == initial_agents

    # Constraints
    W_prev = 0  # Initial waiting pool
    S_prev = 0  # Initial assigned advertisers

    for m in months:
        if m == 0:
            T = A[m]
        else:
            T = A[m] + N[m-1] - F[m-1]

        prob += S[m] <= T * max_adv_per_agent
        prob += W[m] == W_prev + advertiser_data["Advertisers"][m] - S[m] - D[m]
        prob += W[m] >= 0

        if m >= 2:
            prob += D[m] == W[m-2]
            prob += G[m] == S[m-2]
        else:
            prob += D[m] == 0
            prob += G[m] == 0

        prob += N[m] <= A[m]
        prob += F[m] <= A[m]
        prob += T * 10 >= S[m] + S_prev

        if m < n_months - 1:
            prob += A[m+1] == A[m] + N[m] - F[m] + G[m]

        W_prev = W[m]
        S_prev = S[m]

    # Objective Function
    total_cost = lp.lpSum([
        A[m] * monthly_salary + F[m] * firing_cost_perc * annual_salary
        - S[m] * advertiser_data["Avg_Budget"][m] * expected_uplift
        + W[m] * (advertiser_data["Avg_Budget"][m] / 12 * expected_uplift)
        for m in months
    ])
    prob += total_cost

    # Solve the model
    prob.solve()

    # Store results
    results = []
    for m in months:
        results.append({
            "Month": advertiser_data["Month"][m],
            "Available Agents": A[m].varValue,
            "New Hires": N[m].varValue,
            "Fired": F[m].varValue,
            "Assigned Advertisers": S[m].varValue,
            "Waiting Advertisers": W[m].varValue,
            "Dropped Advertisers": D[m].varValue,
            "Graduated Advertisers": G[m].varValue,
            "Total Cost ($)": round(A[m].varValue * monthly_salary + F[m].varValue * firing_cost_perc * annual_salary, 2),
            "Revenue Uplift ($)": round(S[m].varValue * advertiser_data["Avg_Budget"][m] * expected_uplift, 2),
            "Waiting Cost Penalty ($)": round(W[m].varValue * (advertiser_data["Avg_Budget"][m] / 12 * expected_uplift), 2)
        })

    optimization_results[country] = pd.DataFrame(results)

# Concatenate results into one dataframe
final_optimization_df = pd.concat(optimization_results, names=["Country"]).reset_index(level=1, drop=True).reset_index()




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/7ca53fa94c3e4a33942c4e46494b9456-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/7ca53fa94c3e4a33942c4e46494b9456-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 113 COLUMNS
At line 640 RHS
At line 749 BOUNDS
At line 834 ENDATA
Problem MODEL has 108 rows, 84 columns and 310 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.52843e+06 - 0.00 seconds
Cgl0003I 0 fixed, 56 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 56 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 27 tightened bounds, 0 strengthened rows, 0 substitutio

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/8c148ed4386542bf8861ffeb20ec80af-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/8c148ed4386542bf8861ffeb20ec80af-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 113 COLUMNS
At line 640 RHS
At line 749 BOUNDS
At line 834 ENDATA
Problem MODEL has 108 rows, 84 columns and 310 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.41767e+06 - 0.00 seconds
Cgl0003I 0 fixed, 47 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 48 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 30 tightened bounds, 0 strengthened rows, 0 substitutio

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/125a9d2d11bd404e8bda1e15d84c887e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/125a9d2d11bd404e8bda1e15d84c887e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 113 COLUMNS
At line 640 RHS
At line 749 BOUNDS
At line 834 ENDATA
Problem MODEL has 108 rows, 84 columns and 310 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -6.05697e+06 - 0.00 seconds
Cgl0003I 0 fixed, 51 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 54 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 31 tightened bounds, 0 strengthened rows, 0 substituti

In [5]:
final_optimization_df

,Country,Month,Available Agents,New Hires,Fired,Assigned Advertisers,Waiting Advertisers,Dropped Advertisers,Graduated Advertisers,Total Cost ($),Revenue Uplift ($),Waiting Cost Penalty ($)
0,Canada,2025-01,228.0,0.0,113.0,0.0,371.0,0.0,0.0,3920052.00,0.00,188682.72
1,Canada,2025-02,115.0,0.0,56.0,13.0,703.0,0.0,0.0,1952902.33,80436.92,362481.74
2,Canada,2025-03,59.0,0.0,29.0,8.0,678.0,371.0,0.0,1008507.67,52294.68,369331.18
3,Canada,2025-04,30.0,0.0,2.0,0.0,332.0,703.0,13.0,201498.00,0.00,170442.99
4,Canada,2025-05,41.0,0.0,0.0,0.0,24.0,678.0,8.0,208621.67,0.00,12271.77
...,...,...,...,...,...,...,...,...,...,...,...,...
259,Thailand,2025-08,8.0,0.0,0.0,0.0,155.0,4.0,0.0,16192.67,0.00,14966.61
260,Thailand,2025-09,8.0,3.0,0.0,35.0,137.0,88.0,0.0,16192.67,44599.28,14547.86
261,Thailand,2025-10,11.0,0.0,0.0,105.0,2.0,155.0,0.0,22264.92,159950.70,253.89
262,Thailand,2025-11,11.0,0.0,0.0,0.0,0.0,137.0,35.0,22264.92,0.00,0.00


In [6]:
# Define the file path
output_csv_path = "Optimized_Staffing_Results.csv"

# Export the DataFrame to CSV
final_optimization_df.to_csv(output_csv_path, index=False)

print(f"File saved as {output_csv_path}")

File saved as Optimized_Staffing_Results.csv


In [30]:
# Store results for all countries
optimization_results = {}

# Load the processed data from CSV
final_df = pd.read_csv("Final_Processed_Data.csv")

# Iterate through each country
for country in final_df["Country"].unique():
    # Filter data for the specific country
    country_df = final_df[final_df["Country"] == country]

    # Extract parameters
    advertiser_data = country_df[["Month", "Advertisers", "Avg_Budget"]].reset_index(drop=True)
    initial_agents = country_df["Existing_Agent_Count"].iloc[0]
    annual_salary = country_df["Annual_Agent_Salary_USD"].iloc[0]
    monthly_salary = annual_salary / 12

    # Constants
    firing_cost_perc = 0.4
    max_adv_per_agent = 10
    expected_uplift = 0.135
    n_months = len(advertiser_data)

    # Initialize PuLP problem
    prob = lp.LpProblem(f"Dynamic_Staffing_Optimization_{country}", lp.LpMinimize)

    # Decision Variables
    months = list(range(n_months))
    A = lp.LpVariable.dicts("Available_Agents", months, lowBound=0, cat='Integer')
    N = lp.LpVariable.dicts("New_Hires", months, lowBound=0, cat='Integer')
    F = lp.LpVariable.dicts("Fired", months, lowBound=0, cat='Integer')
    S = lp.LpVariable.dicts("Assigned_Advertisers", months, lowBound=0, cat='Integer')
    W = lp.LpVariable.dicts("Waiting_Pool", months, lowBound=0, cat='Integer')
    D = lp.LpVariable.dicts("Dropped_Advertisers", months, lowBound=0, cat='Integer')
    G = lp.LpVariable.dicts("Graduated_Advertisers", months, lowBound=0, cat='Integer')
    T = lp.LpVariable.dicts("Total_Agents", months, lowBound=0, cat='Integer')  # Total Agents in month (need to pay salary)

    # Initial Conditions
    prob += A[0] == initial_agents  # Starting agents
    W_prev = 0  # Initial waiting pool
    S_prev = 0  # Initial assigned advertisers

    # Constraints
    for m in months:

        # 1. Waiting Pool: W[m] = W_prev + new advertisers - assigned - dropped
        prob += W[m] == W_prev + advertiser_data['Advertisers'][m] - S[m] - D[m]
        prob += W[m] >= 0

        # 2. Dropped Advertisers: Those waiting for more than 60 days
        if m >= 2:
            prob += D[m] == W[m-2]  # Drop advertisers who have been waiting for 2 months
        else:
            prob += D[m] == 0  # No one is dropped in the first two months

        # 3. Graduated Advertisers: Advertisers exit after 60 days of support
        if m >= 2:
            prob += G[m] == S[m-2]  # Graduates advertisers assigned two months ago
        else:
            prob += G[m] == 0  # No one graduates in the first two months

        # 4. Hiring/Firing Constraints
        prob += N[m] <= A[m]
        prob += F[m] <= A[m]

        # 5. Required Agents: A[m] * 10 >= S[m] + S_prev (eliminates division)
        prob += A[m] * 10 >= S[m] + S_prev  # Ensures sufficient capacity for current and past advertisers

        # Update next month's available agents
        if m < n_months - 1:
            prob += A[m+1] == A[m] + N[m] - F[m]

        # Update variables for next iteration
        W_prev = W[m]
        S_prev = S[m]

        # Update Total Agents (Can support advertisers + cannot support advertisers)
        if m == 0:
            prob += T[m] == A[m] + N[m]
        else:
            prob += T[m] == T[m-1] + N[m] - F[m-1]

    # Objective Function (Minimize Costs + Waiting Time Penalty)
    total_cost = lp.lpSum(
        [A[m] * monthly_salary + F[m] * firing_cost_perc * annual_salary
        - S[m] * advertiser_data['Avg_Budget'][m] * expected_uplift
        + W[m] * (advertiser_data['Avg_Budget'][m] / 12 * expected_uplift)  # Waiting penalty
        for m in months]
    )
    prob += total_cost

    # Solve
    prob.solve()

    # Store results
    results = []

    for m in months:
        results.append({
            'Month': advertiser_data['Month'][m],
            'Available Agents': A[m].varValue,
            'New Hires': N[m].varValue,
            'Fired': F[m].varValue,
            'Total Agents':T[m].varValue,
            'Assigned Advertisers': S[m].varValue,
            'Waiting Advertisers': W[m].varValue,
            'Dropped Advertisers': D[m].varValue,
            'Graduated Advertisers': G[m].varValue,
            'Total Cost ($)': round(T[m].varValue * monthly_salary + F[m].varValue * firing_cost_perc * annual_salary, 2),
            'Revenue Uplift ($)': round(S[m].varValue * advertiser_data['Avg_Budget'][m] * expected_uplift, 2),
            'Waiting Cost Penalty ($)': round(W[m].varValue * (advertiser_data['Avg_Budget'][m] / 12 * expected_uplift), 2),
            "Net Profit ($)": (round(S[m].varValue * advertiser_data["Avg_Budget"][m] * expected_uplift, 2)) - (round(T[m].varValue * monthly_salary + F[m].varValue * firing_cost_perc * annual_salary, 2))   # New column added
    })

    optimization_results[country] = pd.DataFrame(results)

# Concatenate results into one dataframe
final_optimization_df = pd.concat(optimization_results, names=["Country"]).reset_index(level=1, drop=True).reset_index()

    




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/d8c0e9c1891447e784117db35720e930-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/d8c0e9c1891447e784117db35720e930-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 113 COLUMNS
At line 632 RHS
At line 741 BOUNDS
At line 838 ENDATA
Problem MODEL has 108 rows, 96 columns and 278 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -2.02184e+07 - 0.00 seconds
Cgl0003I 0 fixed, 20 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 16 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 12 tightened bounds, 0 strengthened rows, 0 substituti

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/d32bc9f3afab4f2b8a4a26efae0fc901-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hy/frf0zxvn1k5_c7zh7z63995r0000gn/T/d32bc9f3afab4f2b8a4a26efae0fc901-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 113 COLUMNS
At line 632 RHS
At line 741 BOUNDS
At line 838 ENDATA
Problem MODEL has 108 rows, 96 columns and 278 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -2.90532e+07 - 0.00 seconds
Cgl0003I 0 fixed, 20 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 16 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 12 tightened bounds, 0 strengthened rows, 0 substituti

In [32]:
final_optimization_df

,Country,Month,Available Agents,New Hires,Fired,Total Agents,Assigned Advertisers,Waiting Advertisers,Dropped Advertisers,Graduated Advertisers,Total Cost ($),Revenue Uplift ($),Waiting Cost Penalty ($),Net Profit ($)
0,Canada,2025-01,228.0,0.0,155.0,228.0,371.0,0.0,0.0,0.0,4945860.00,2264192.60,0.00,-2681667.40
1,Canada,2025-02,73.0,0.0,0.0,73.0,345.0,0.0,0.0,0.0,371448.33,2134671.98,0.00,1763223.65
2,Canada,2025-03,73.0,0.0,0.0,73.0,354.0,0.0,0.0,371.0,371448.33,2314039.59,0.00,1942591.26
3,Canada,2025-04,73.0,0.0,0.0,73.0,357.0,0.0,0.0,345.0,371448.33,2199330.63,0.00,1827882.30
4,Canada,2025-05,73.0,0.0,0.0,73.0,370.0,0.0,0.0,354.0,371448.33,2270277.45,0.00,1898829.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Thailand,2025-08,18.0,0.0,0.0,18.0,71.0,0.0,0.0,80.0,36433.50,82268.06,0.00,45834.56
260,Thailand,2025-09,18.0,5.0,0.0,23.0,105.0,0.0,0.0,84.0,46553.92,133797.83,0.00,87243.91
261,Thailand,2025-10,23.0,0.0,0.0,23.0,125.0,0.0,0.0,71.0,46553.92,190417.50,0.00,143863.58
262,Thailand,2025-11,23.0,3.0,0.0,26.0,105.0,30.0,0.0,105.0,52626.17,137979.45,3285.23,85353.28


In [33]:
# Define the file path
output_csv_path = "Optimized_Staffing_Results_3.csv"

# Export the DataFrame to CSV
final_optimization_df.to_csv(output_csv_path, index=False)

print(f"File saved as {output_csv_path}")

File saved as Optimized_Staffing_Results_3.csv
